In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re

%matplotlib inline

In [2]:
final_data = pd.read_csv('../data/teis_final_data.csv')

In [3]:
final_data.columns = [
    col.strip().lower().replace(' ', '_').replace('.', '_') 
    for col in 
    final_data.columns]
final_data.dtypes

child_id                       int64
notification_date             object
fiscal_year                   object
notification_month            object
tenn_region                   object
fiscal_year_1                float64
third_dob                     object
late_referral                float64
qtr                          float64
poe                           object
dob                           object
county_name                   object
county_ses                    object
child_phase                   object
active                        object
service_coordinator           object
parent_consent                object
initial_eligibility           object
initial_eligibility_date      object
initial_ifsp_date             object
latest_ifsp                   object
exit_reason                   object
exit_date                     object
referral_source_type_name     object
year                           int64
dtype: object

In [4]:
#keeping only the columns necessary to answer the question
age = final_data[['notification_date','dob', 'exit_date','poe','child_id','county_name','county_ses']]
age

,notification_date,dob,exit_date,poe,child_id,county_name,county_ses
0,2018-01-20,2007-07-07,2010-01-25,GN,104085,Davidson,Competitive
1,2016-08-17,2013-10-15,2016-08-31,UC,350065,Jackson,At-Risk
2,2016-07-12,2013-11-13,2016-08-25,NW,351755,Madison,Transitional
3,2016-09-14,2013-10-01,2016-09-15,FT,353800,Hawkins,At-Risk
4,2016-08-05,2013-12-04,2016-12-03,UC,353805,Putnam,Transitional
...,...,...,...,...,...,...,...
115519,2022-07-01,2020-05-20,NaN,GN,531592,Montgomery,Transitional
115520,2022-07-01,2021-12-07,NaN,GN,531593,Davidson,Competitive
115521,2022-07-01,2022-06-21,NaN,GN,531594,Montgomery,Transitional
115522,2022-07-01,2022-06-17,NaN,GN,531595,Davidson,Competitive


In [5]:
#checking columns type
age.dtypes

notification_date    object
dob                  object
exit_date            object
poe                  object
child_id              int64
county_name          object
county_ses           object
dtype: object

In [6]:
# trasforming notification date, DOB, and Exit date from object into datetime
age['notification_date']=pd.to_datetime(age['notification_date'])
age['dob']=pd.to_datetime(age['dob'])
age['exit_date']=pd.to_datetime(age['exit_date'])

C:\Users\dkham\AppData\Local\Temp/ipykernel_150860/3969099700.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age['notification_date']=pd.to_datetime(age['notification_date'])
C:\Users\dkham\AppData\Local\Temp/ipykernel_150860/3969099700.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age['dob']=pd.to_datetime(age['dob'])
C:\Users\dkham\AppData\Local\Temp/ipykernel_150860/3969099700.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [7]:
age.dtypes

notification_date    datetime64[ns]
dob                  datetime64[ns]
exit_date            datetime64[ns]
poe                          object
child_id                      int64
county_name                  object
county_ses                   object
dtype: object

In [8]:
# creating a column for kids ages from the notification date column
age['kids_notification_age'] = (age['notification_date'] - age['dob'])
age

C:\Users\dkham\AppData\Local\Temp/ipykernel_150860/3244466132.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age['kids_notification_age'] = (age['notification_date'] - age['dob'])


,notification_date,dob,exit_date,poe,child_id,county_name,county_ses,kids_notification_age
0,2018-01-20,2007-07-07,2010-01-25,GN,104085,Davidson,Competitive,3850 days
1,2016-08-17,2013-10-15,2016-08-31,UC,350065,Jackson,At-Risk,1037 days
2,2016-07-12,2013-11-13,2016-08-25,NW,351755,Madison,Transitional,972 days
3,2016-09-14,2013-10-01,2016-09-15,FT,353800,Hawkins,At-Risk,1079 days
4,2016-08-05,2013-12-04,2016-12-03,UC,353805,Putnam,Transitional,975 days
...,...,...,...,...,...,...,...,...
115519,2022-07-01,2020-05-20,NaT,GN,531592,Montgomery,Transitional,772 days
115520,2022-07-01,2021-12-07,NaT,GN,531593,Davidson,Competitive,206 days
115521,2022-07-01,2022-06-21,NaT,GN,531594,Montgomery,Transitional,10 days
115522,2022-07-01,2022-06-17,NaT,GN,531595,Davidson,Competitive,14 days


In [9]:
# creating a column for kids ages from the exit date column
age['kids_exit_age'] = (age['exit_date'] - age['dob'])
age

C:\Users\dkham\AppData\Local\Temp/ipykernel_150860/714906495.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age['kids_exit_age'] = (age['exit_date'] - age['dob'])


,notification_date,dob,exit_date,poe,child_id,county_name,county_ses,kids_notification_age,kids_exit_age
0,2018-01-20,2007-07-07,2010-01-25,GN,104085,Davidson,Competitive,3850 days,933 days
1,2016-08-17,2013-10-15,2016-08-31,UC,350065,Jackson,At-Risk,1037 days,1051 days
2,2016-07-12,2013-11-13,2016-08-25,NW,351755,Madison,Transitional,972 days,1016 days
3,2016-09-14,2013-10-01,2016-09-15,FT,353800,Hawkins,At-Risk,1079 days,1080 days
4,2016-08-05,2013-12-04,2016-12-03,UC,353805,Putnam,Transitional,975 days,1095 days
...,...,...,...,...,...,...,...,...,...
115519,2022-07-01,2020-05-20,NaT,GN,531592,Montgomery,Transitional,772 days,NaT
115520,2022-07-01,2021-12-07,NaT,GN,531593,Davidson,Competitive,206 days,NaT
115521,2022-07-01,2022-06-21,NaT,GN,531594,Montgomery,Transitional,10 days,NaT
115522,2022-07-01,2022-06-17,NaT,GN,531595,Davidson,Competitive,14 days,NaT


In [10]:
# transforming days in months
age['month_notification_age'] = age['kids_exit_age'].dt.days/30
age

C:\Users\dkham\AppData\Local\Temp/ipykernel_150860/523736473.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age['month_notification_age'] = age['kids_exit_age'].dt.days/30


,notification_date,dob,exit_date,poe,child_id,county_name,county_ses,kids_notification_age,kids_exit_age,month_notification_age
0,2018-01-20,2007-07-07,2010-01-25,GN,104085,Davidson,Competitive,3850 days,933 days,31.100000
1,2016-08-17,2013-10-15,2016-08-31,UC,350065,Jackson,At-Risk,1037 days,1051 days,35.033333
2,2016-07-12,2013-11-13,2016-08-25,NW,351755,Madison,Transitional,972 days,1016 days,33.866667
3,2016-09-14,2013-10-01,2016-09-15,FT,353800,Hawkins,At-Risk,1079 days,1080 days,36.000000
4,2016-08-05,2013-12-04,2016-12-03,UC,353805,Putnam,Transitional,975 days,1095 days,36.500000
...,...,...,...,...,...,...,...,...,...,...
115519,2022-07-01,2020-05-20,NaT,GN,531592,Montgomery,Transitional,772 days,NaT,NaN
115520,2022-07-01,2021-12-07,NaT,GN,531593,Davidson,Competitive,206 days,NaT,NaN
115521,2022-07-01,2022-06-21,NaT,GN,531594,Montgomery,Transitional,10 days,NaT,NaN
115522,2022-07-01,2022-06-17,NaT,GN,531595,Davidson,Competitive,14 days,NaT,NaN


In [11]:
# transforming days in years
age['years_notification_age'] = age['kids_exit_age'].dt.days/365
age

C:\Users\dkham\AppData\Local\Temp/ipykernel_150860/2564336644.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age['years_notification_age'] = age['kids_exit_age'].dt.days/365


,notification_date,dob,exit_date,poe,child_id,county_name,county_ses,kids_notification_age,kids_exit_age,month_notification_age,years_notification_age
0,2018-01-20,2007-07-07,2010-01-25,GN,104085,Davidson,Competitive,3850 days,933 days,31.100000,2.556164
1,2016-08-17,2013-10-15,2016-08-31,UC,350065,Jackson,At-Risk,1037 days,1051 days,35.033333,2.879452
2,2016-07-12,2013-11-13,2016-08-25,NW,351755,Madison,Transitional,972 days,1016 days,33.866667,2.783562
3,2016-09-14,2013-10-01,2016-09-15,FT,353800,Hawkins,At-Risk,1079 days,1080 days,36.000000,2.958904
4,2016-08-05,2013-12-04,2016-12-03,UC,353805,Putnam,Transitional,975 days,1095 days,36.500000,3.000000
...,...,...,...,...,...,...,...,...,...,...,...
115519,2022-07-01,2020-05-20,NaT,GN,531592,Montgomery,Transitional,772 days,NaT,NaN,NaN
115520,2022-07-01,2021-12-07,NaT,GN,531593,Davidson,Competitive,206 days,NaT,NaN,NaN
115521,2022-07-01,2022-06-21,NaT,GN,531594,Montgomery,Transitional,10 days,NaT,NaN,NaN
115522,2022-07-01,2022-06-17,NaT,GN,531595,Davidson,Competitive,14 days,NaT,NaN,NaN


In [12]:
# using only kids that are between 0 and 3 years old only (cleaning any outlier)
age = age[age['kids_notification_age'] < '1096 days']
age

,notification_date,dob,exit_date,poe,child_id,county_name,county_ses,kids_notification_age,kids_exit_age,month_notification_age,years_notification_age
1,2016-08-17,2013-10-15,2016-08-31,UC,350065,Jackson,At-Risk,1037 days,1051 days,35.033333,2.879452
2,2016-07-12,2013-11-13,2016-08-25,NW,351755,Madison,Transitional,972 days,1016 days,33.866667,2.783562
3,2016-09-14,2013-10-01,2016-09-15,FT,353800,Hawkins,At-Risk,1079 days,1080 days,36.000000,2.958904
4,2016-08-05,2013-12-04,2016-12-03,UC,353805,Putnam,Transitional,975 days,1095 days,36.500000,3.000000
5,2016-09-29,2013-10-09,2016-09-29,MD,353900,Shelby,Transitional,1086 days,1086 days,36.200000,2.975342
...,...,...,...,...,...,...,...,...,...,...,...
115519,2022-07-01,2020-05-20,NaT,GN,531592,Montgomery,Transitional,772 days,NaT,NaN,NaN
115520,2022-07-01,2021-12-07,NaT,GN,531593,Davidson,Competitive,206 days,NaT,NaN,NaN
115521,2022-07-01,2022-06-21,NaT,GN,531594,Montgomery,Transitional,10 days,NaT,NaN,NaN
115522,2022-07-01,2022-06-17,NaT,GN,531595,Davidson,Competitive,14 days,NaT,NaN,NaN


In [13]:
# turning exit date into monthly
age['month_exit_age'] = age['kids_exit_age'].dt.days/30
age

C:\Users\dkham\AppData\Local\Temp/ipykernel_150860/62103522.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age['month_exit_age'] = age['kids_exit_age'].dt.days/30


,notification_date,dob,exit_date,poe,child_id,county_name,county_ses,kids_notification_age,kids_exit_age,month_notification_age,years_notification_age,month_exit_age
1,2016-08-17,2013-10-15,2016-08-31,UC,350065,Jackson,At-Risk,1037 days,1051 days,35.033333,2.879452,35.033333
2,2016-07-12,2013-11-13,2016-08-25,NW,351755,Madison,Transitional,972 days,1016 days,33.866667,2.783562,33.866667
3,2016-09-14,2013-10-01,2016-09-15,FT,353800,Hawkins,At-Risk,1079 days,1080 days,36.000000,2.958904,36.000000
4,2016-08-05,2013-12-04,2016-12-03,UC,353805,Putnam,Transitional,975 days,1095 days,36.500000,3.000000,36.500000
5,2016-09-29,2013-10-09,2016-09-29,MD,353900,Shelby,Transitional,1086 days,1086 days,36.200000,2.975342,36.200000
...,...,...,...,...,...,...,...,...,...,...,...,...
115519,2022-07-01,2020-05-20,NaT,GN,531592,Montgomery,Transitional,772 days,NaT,NaN,NaN,NaN
115520,2022-07-01,2021-12-07,NaT,GN,531593,Davidson,Competitive,206 days,NaT,NaN,NaN,NaN
115521,2022-07-01,2022-06-21,NaT,GN,531594,Montgomery,Transitional,10 days,NaT,NaN,NaN,NaN
115522,2022-07-01,2022-06-17,NaT,GN,531595,Davidson,Competitive,14 days,NaT,NaN,NaN,NaN


In [14]:
# turning exit date into year
age['years_exit_age'] = age['kids_exit_age'].dt.days/365
age

C:\Users\dkham\AppData\Local\Temp/ipykernel_150860/120777978.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age['years_exit_age'] = age['kids_exit_age'].dt.days/365


,notification_date,dob,exit_date,poe,child_id,county_name,county_ses,kids_notification_age,kids_exit_age,month_notification_age,years_notification_age,month_exit_age,years_exit_age
1,2016-08-17,2013-10-15,2016-08-31,UC,350065,Jackson,At-Risk,1037 days,1051 days,35.033333,2.879452,35.033333,2.879452
2,2016-07-12,2013-11-13,2016-08-25,NW,351755,Madison,Transitional,972 days,1016 days,33.866667,2.783562,33.866667,2.783562
3,2016-09-14,2013-10-01,2016-09-15,FT,353800,Hawkins,At-Risk,1079 days,1080 days,36.000000,2.958904,36.000000,2.958904
4,2016-08-05,2013-12-04,2016-12-03,UC,353805,Putnam,Transitional,975 days,1095 days,36.500000,3.000000,36.500000,3.000000
5,2016-09-29,2013-10-09,2016-09-29,MD,353900,Shelby,Transitional,1086 days,1086 days,36.200000,2.975342,36.200000,2.975342
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115519,2022-07-01,2020-05-20,NaT,GN,531592,Montgomery,Transitional,772 days,NaT,NaN,NaN,NaN,NaN
115520,2022-07-01,2021-12-07,NaT,GN,531593,Davidson,Competitive,206 days,NaT,NaN,NaN,NaN,NaN
115521,2022-07-01,2022-06-21,NaT,GN,531594,Montgomery,Transitional,10 days,NaT,NaN,NaN,NaN,NaN
115522,2022-07-01,2022-06-17,NaT,GN,531595,Davidson,Competitive,14 days,NaT,NaN,NaN,NaN,NaN


In [16]:
#dropping duplicates to get only referrals
age_referrals = age.drop_duplicates('child_id')
age_referrals

,notification_date,dob,exit_date,poe,child_id,county_name,county_ses,kids_notification_age,kids_exit_age,month_notification_age,years_notification_age,month_exit_age,years_exit_age
1,2016-08-17,2013-10-15,2016-08-31,UC,350065,Jackson,At-Risk,1037 days,1051 days,35.033333,2.879452,35.033333,2.879452
2,2016-07-12,2013-11-13,2016-08-25,NW,351755,Madison,Transitional,972 days,1016 days,33.866667,2.783562,33.866667,2.783562
3,2016-09-14,2013-10-01,2016-09-15,FT,353800,Hawkins,At-Risk,1079 days,1080 days,36.000000,2.958904,36.000000,2.958904
4,2016-08-05,2013-12-04,2016-12-03,UC,353805,Putnam,Transitional,975 days,1095 days,36.500000,3.000000,36.500000,3.000000
5,2016-09-29,2013-10-09,2016-09-29,MD,353900,Shelby,Transitional,1086 days,1086 days,36.200000,2.975342,36.200000,2.975342
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115519,2022-07-01,2020-05-20,NaT,GN,531592,Montgomery,Transitional,772 days,NaT,NaN,NaN,NaN,NaN
115520,2022-07-01,2021-12-07,NaT,GN,531593,Davidson,Competitive,206 days,NaT,NaN,NaN,NaN,NaN
115521,2022-07-01,2022-06-21,NaT,GN,531594,Montgomery,Transitional,10 days,NaT,NaN,NaN,NaN,NaN
115522,2022-07-01,2022-06-17,NaT,GN,531595,Davidson,Competitive,14 days,NaT,NaN,NaN,NaN,NaN


In [17]:
# referrals only
age_referrals.describe()

,child_id,kids_notification_age,kids_exit_age,month_notification_age,years_notification_age,month_exit_age,years_exit_age
count,96731.000000,96731,78027,78027.000000,78027.000000,78027.000000,78027.000000
mean,465546.444728,471 days 17:02:47.214233288,669 days 16:07:56.142873616,22.322406,1.834718,22.322406,1.834718
std,38158.318297,322 days 15:46:38.113216064,380 days 11:13:37.985395848,12.682260,1.042378,12.682260,1.042378
min,350065.000000,-178 days +00:00:00,0 days 00:00:00,0.000000,0.000000,0.000000,0.000000
25%,430722.500000,133 days 00:00:00,317 days 00:00:00,10.566667,0.868493,10.566667,0.868493
50%,467097.000000,519 days 00:00:00,755 days 00:00:00,25.166667,2.068493,25.166667,2.068493
75%,497839.500000,745 days 00:00:00,1075 days 00:00:00,35.833333,2.945205,35.833333,2.945205
max,531596.000000,1095 days 00:00:00,1114 days 00:00:00,37.133333,3.052055,37.133333,3.052055


In [ ]:
# re referrals
age_reref = age[age.duplicated(subset = "Child ID", keep= 'first')]
#age_reref = age.groupby(["Child ID"]).nunique()
age_reref.describe()